In [1]:
# Fix randomness and hide warnings
seed = 42

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ['MPLCONFIGDIR'] = os.getcwd()+'/configs/'

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

import numpy as np
np.random.seed(seed)

import logging

import random
random.seed(seed)

In [2]:
# Import tensorflow
import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl
from tensorflow.keras import initializers
tf.autograph.set_verbosity(0)
tf.get_logger().setLevel(logging.ERROR)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)
print(tf.__version__)

2.12.0


In [14]:
# Import other libraries
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
from sklearn.utils import class_weight
import seaborn as sns

In [15]:
TRAIN_DIR="/kaggle/input/datadirectory/data/training_folder"
VAL_DIR="/kaggle/input/datadirectory/data/validation_folder"
TEST_DIR="/kaggle/input/datadirectory/data/test_folder"
IMG_SHAPE=(96,96)
BATCH_SIZE=32
NUM_CLASSES=2
INPUT_SHAPE = (*IMG_SHAPE,3)

In [16]:
import keras_tuner as kt
import time
from pathlib import Path

In [17]:
LOG_DIR = '/kaggle/working/kerastuner_logs'

import os
os.makedirs(LOG_DIR, exist_ok=True)

# **Data Generator**

In [18]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# train generator with augmentation
train_image_gen  = ImageDataGenerator(rotation_range=40,
                                      width_shift_range=0.2,
                                      height_shift_range=0.2,
                                      zoom_range=[0.5,1.5],
                                      brightness_range=[0.5,1.5],
                                      shear_range=0.2,
                                      vertical_flip=True,
                                      horizontal_flip=True,
                                      fill_mode='reflect',
                                      )

validation_image_gen = ImageDataGenerator()

train_dataset = train_image_gen.flow_from_directory(directory=TRAIN_DIR,
                                                    target_size=IMG_SHAPE,
                                                   # color_mode='rgb',
                                                    classes=None,
                                                    class_mode='binary',
                                                    batch_size=BATCH_SIZE,
                                                    shuffle=True,
                                                    seed=seed,
                                                    )

validation_dataset = validation_image_gen.flow_from_directory(directory=VAL_DIR,
                                                              target_size=IMG_SHAPE,
                                                             # color_mode='rgb',
                                                              classes=None,
                                                              class_mode='binary',
                                                              batch_size=BATCH_SIZE,
                                                              shuffle=False,
                                                              seed=seed,
                                                              )

Found 3928 images belonging to 2 classes.
Found 437 images belonging to 2 classes.


In [19]:
class_weights = class_weight.compute_class_weight(class_weight='balanced',
                                                  classes=np.unique(train_dataset.classes),
                                                  y=train_dataset.classes)

class_weights = dict(zip(np.unique(train_dataset.classes), class_weights))
class_weights

{0: 0.7925746569814366, 1: 1.3544827586206896}

# **Tuning MobileNetV2**

In [20]:
conv_base  = tfk.applications.MobileNetV2(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224,3)
)
conv_base.trainable = False

In [21]:
def model_builder(hp):

    inputs = tfk.Input(shape=INPUT_SHAPE)

    x = tfkl.Resizing(224, 224, interpolation='bicubic')(inputs)
    x = tfk.applications.mobilenet.preprocess_input(x)
    x = conv_base(x)

    x = tfkl.GlobalAveragePooling2D()(x)
    
    # best activation
    hp_activation = hp.Choice('activation', values=['relu','tanh'])
    
    # best number of layers
    for i in range(hp.Int('n_layers',1,4)):
        # best number of neurons
        x = tfkl.Dense(units=hp.Int(f'units_{i}', min_value=128, max_value=512, step=128))(x)
        x = tfkl.BatchNormalization()(x)
        x = tfkl.Activation(hp_activation)(x)
        x = tfkl.Dropout(0.3)(x)


    outputs = tfkl.Dense(NUM_CLASSES, activation='softmax', kernel_initializer=tfk.initializers.GlorotUniform(seed=seed))(x)

    model = tfk.Model(inputs, outputs)
    
    # best lr
    # hp_learning_rate = hp.Choice('learning_rate',values=[1e-2,1e-3,1e-4])

    model.compile(loss=tfk.losses.SparseCategoricalCrossentropy(),
                  optimizer=tfk.optimizers.Adam(learning_rate=1e-3), metrics=['accuracy'])

    return model

In [22]:
tuner_mobilenet2 = kt.Hyperband(
    model_builder,
    objective='val_accuracy',
    max_epochs=10,
    directory=LOG_DIR,
    project_name='MobileNetV2')

tuner_mobilenet2.search(train_dataset,
             epochs=50,
             validation_data=validation_dataset,
             class_weight=class_weights,
             callbacks=[tfk.callbacks.EarlyStopping(monitor='val_loss', patience=5)])

Trial 30 Complete [00h 03m 19s]
val_accuracy: 0.7711670398712158

Best val_accuracy So Far: 0.8192219734191895
Total elapsed time: 00h 41m 24s


In [23]:
tuner_mobilenet2.search_space_summary()

Search space summary
Default search space size: 6
activation (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh'], 'ordered': False}
n_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 4, 'step': 1, 'sampling': 'linear'}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 128, 'max_value': 512, 'step': 128, 'sampling': 'linear'}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 128, 'max_value': 512, 'step': 128, 'sampling': 'linear'}
units_2 (Int)
{'default': None, 'conditions': [], 'min_value': 128, 'max_value': 512, 'step': 128, 'sampling': 'linear'}
units_3 (Int)
{'default': None, 'conditions': [], 'min_value': 128, 'max_value': 512, 'step': 128, 'sampling': 'linear'}


In [24]:
best_hp = tuner_mobilenet2.get_best_hyperparameters()[0]
best_hp.values

{'activation': 'relu',
 'n_layers': 1,
 'units_0': 128,
 'tuner/epochs': 10,
 'tuner/initial_epoch': 4,
 'tuner/bracket': 2,
 'tuner/round': 2,
 'tuner/trial_id': '0013',
 'units_1': 512,
 'units_2': 256,
 'units_3': 256}

In [25]:
model = tuner_mobilenet2.hypermodel.build(best_hp)
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 96, 96, 3)]       0         
                                                                 
 resizing_1 (Resizing)       (None, 224, 224, 3)       0         
                                                                 
 tf.math.truediv_1 (TFOpLamb  (None, 224, 224, 3)      0         
 da)                                                             
                                                                 
 tf.math.subtract_1 (TFOpLam  (None, 224, 224, 3)      0         
 bda)                                                            
                                                                 
 mobilenetv2_1.00_224 (Funct  (None, 7, 7, 1280)       2257984   
 ional)                                                          
                                                           

In [27]:
stop_early = tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=15, restore_best_weights=True)
plateau = tfk.callbacks.ReduceLROnPlateau(monitor='val_accuracy',patience=5,factor=0.85,mode='auto',min_lr=1e-5)

# Train the model
history = model.fit(
    train_dataset, # We need to apply the preprocessing thought for the MobileNetV2 network
    batch_size = 64,
    epochs = 120,
    validation_data = validation_dataset, # We need to apply the preprocessing thought for the MobileNetV2 network
    class_weight=class_weights,
    callbacks = [stop_early, plateau]
).history

Epoch 1/120
123/123 [==============================] - 31s 221ms/step - loss: 0.7169 - accuracy: 0.6622 - val_loss: 0.5732 - val_accuracy: 0.7391 - lr: 0.0010
Epoch 2/120
123/123 [==============================] - 21s 169ms/step - loss: 0.5870 - accuracy: 0.7070 - val_loss: 0.4844 - val_accuracy: 0.7803 - lr: 0.0010
Epoch 3/120
123/123 [==============================] - 19s 154ms/step - loss: 0.5625 - accuracy: 0.7207 - val_loss: 0.4949 - val_accuracy: 0.7666 - lr: 0.0010
Epoch 4/120
123/123 [==============================] - 19s 156ms/step - loss: 0.5486 - accuracy: 0.7138 - val_loss: 0.4511 - val_accuracy: 0.7963 - lr: 0.0010
Epoch 5/120
123/123 [==============================] - 19s 154ms/step - loss: 0.5148 - accuracy: 0.7406 - val_loss: 0.4510 - val_accuracy: 0.7918 - lr: 0.0010
Epoch 6/120
123/123 [==============================] - 19s 153ms/step - loss: 0.5223 - accuracy: 0.7296 - val_loss: 0.4391 - val_accuracy: 0.7826 - lr: 0.0010
Epoch 7/120
123/123 [=========================